# MultiHot Sparse Encoding

This tests are based on the previous Error Correction Codes. For those I still don't find a way of easy deconding from a neural network, but there should be something there.

In this study I make an encoding for UTF-8 that can manage all the possible (valid) codes. 

The idea is to reduce the input space, being able to encode any existing text with a finite set that should be smaller than current one-hot encoder approaches. This should provide a universal first layer for text encoding.

The next layers I have an idea on how to work with them, but for the moment let's make a first one that actually works.


The first step is to build an **Overfitted** autoencoder, this is just to validate that the codes are feasible.

A couple of sources:

- [UTF-8 complete table](https://www.utf8-chartable.de/unicode-utf8-table.pl)
- [UTF-8 Wikipedia](https://en.wikipedia.org/wiki/UTF-8)


In [1]:
import numpy as np
import pickle
import torch
from utf8_encoder import *

Since the entire utf-8 univers is NOT the entire $2^{32}$ domain, but there are limitations explained in [the utf-8 description](https://en.wikipedia.org/wiki/UTF-8)

| Number of bytes | Bits for code point | First code point | Last code point | Byte 1   | Byte 2   | Byte 3   | Byte 4   |
|----------------|--------------------|-----------------|----------------|----------|----------|----------|----------|
| 1              | 7                  | U+0000          | U+007F         | 0xxxxxxx |          |          |          |
| 2              | 11                 | U+0080          | U+07FF         | 110xxxxx | 10xxxxxx |          |          |
| 3              | 16                 | U+0800          | U+FFFF         | 1110xxxx | 10xxxxxx | 10xxxxxx |          |
| 4              | 21                 | U+10000         | U+10FFFF       | 11110xxx | 10xxxxxx | 10xxxxxx | 10xxxxxx |

I'll then compute different table parts and do an append when needed

The thing is that the number of elements in the table should be at most $2^{21}$, I need to create a sort of index that can handle the 4 cases.
It seems I'll have to create 4 different conversion tables.




In fact ... it seems that I can just chunk the utf-8 value in chunks and do one-hot per different parts:
- there are only 4 segment ranges, that can be coded in one-hot also add there either hamming or other ECC
- the largest value is for 7 bits -> 128 values
- the others contain 6 bits -> 64 values
The prefix of each can be taken away and replaced by the initial one-hot

So a complete code would be:  $ 4 + 128 + 64 + 64 + 64 = 324 $

Instead of having dimension 1,112,064 to encode any utf-8 value.

The encoder is  much simpler than I thought for this case, later I can add ECC for each, knowing that there is only one active bit in each row, this makes the task easier.

This embedding can stil be reduced but should be sparse enough already to make a good input

In [2]:
4 + 128 + 64 + 64 + 64 

324

In [3]:
# number of parameters for a one-hot by chunks encoding:
chunk_sizes = [4, 5, 6, 8, 12]
n_params = []
for c in chunk_sizes:
    n_params.append((c, (32 // c) * 2**c))

In [4]:
n_params

[(4, 128), (5, 192), (6, 320), (8, 1024), (12, 8192)]

In [5]:
import torch

emd = torch.nn.Embedding(2**10, 300)

In [6]:
model_parameters = filter(lambda p: p.requires_grad, emd.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])


In [7]:
params

307200

In [8]:
# from https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/7
# counting the number of (trainable) parameters of a pytorch model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
count_parameters(emd)

307200

The embedding layer is a fully connected layer ... this means a LOT of parameters

To be able to do an effective one-hot of all utf-8 would be:

In [10]:
for i in [50,100,200,300]:
    print(i, 1112064 * i)

50 55603200
100 111206400
200 222412800
300 333619200


Which means I don't want to train that layer ... it would not even fit in my GPU

There is another thing, the embedding layer learns from the sample input, this means that it will ignore all values that don't appear or are underrepresented (a know issue). My goal is to deal with this with meta-learning techniques, but always being able to keep adding new inputs.


In [11]:
tables = create_tables(segments=3)
# ctables = add_mappings(tables)

0 0 192 128
0 1 192 129
0 2 192 130
0 3 192 131
0 4 192 132
0 5 192 133
0 6 192 134
0 7 192 135
0 8 192 136
0 9 192 137
0 10 192 138
0 11 192 139
0 12 192 140
0 13 192 141
0 14 192 142
0 15 192 143
0 16 192 144
0 17 192 145
0 18 192 146
0 19 192 147
0 20 192 148
0 21 192 149
0 22 192 150
0 23 192 151
0 24 192 152
0 25 192 153
0 26 192 154
0 27 192 155
0 28 192 156
0 29 192 157
0 30 192 158
0 31 192 159
0 32 192 160
0 33 192 161
0 34 192 162
0 35 192 163
0 36 192 164
0 37 192 165
0 38 192 166
0 39 192 167
0 40 192 168
0 41 192 169
0 42 192 170
0 43 192 171
0 44 192 172
0 45 192 173
0 46 192 174
0 47 192 175
0 48 192 176
0 49 192 177
0 50 192 178
0 51 192 179
0 52 192 180
0 53 192 181
0 54 192 182
0 55 192 183
0 56 192 184
0 57 192 185
0 58 192 186
0 59 192 187
0 60 192 188
0 61 192 189
0 62 192 190
0 63 192 191
1 0 193 128
1 1 193 129


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 0: invalid start byte

In [ ]:
# tables = create_tables()  # 4 segments by default
# if the previous line is executed gives:
# number of codes =  1107904
# number of code_exceptions =  790656


In [ ]:
np.save("utf8_code_matrix_3seg", tables[0])

In [ ]:
save_obj(tables[1], "txt2code_3seg")
save_obj(tables[2], "code2txt_3seg")
save_obj(tables[3], "txt2num_3seg")
save_obj(tables[4], "num2txt_3seg")

In [ ]:
t2c = tables[1]
c2t = tables[2]
n2t = tables[4]
t2n = tables[3]

In [ ]:
# take(50, c2t.items())

In [ ]:
# checking that all the tables have the right number of codes
tables[0].shape, len(t2n.keys()), len(n2t.keys()), len(t2c.keys()), len(c2t.keys())

the text2code and text to num are bigger than the backward reference, this is because of the added mappings. 
the added mappings are for the special codes I count on using the first 20 elements of the utf-8 0000 -> 0020 which are special indicators as NUL, SOH, STX, ETX, EOT, ACK, ....

Mappings:
- *\<start\>* -> STX = 0002 Start of Transaction
- *\<end\>* -> ETX = 0003 End of Transaction
- *\<unknown\>* = *\<unk\>* -> NAK = 0015

The reverse mappings go back only to the new name of the elements so that is why the difference is (and must be) the number of special mappings added = 3

Although Wikipedia says:

'''
  UTF-8 is a variable width character encoding capable of encoding all 1,112,064 valid code points in Unicode using one to four 8-bit bytes.
'''

We have managed to encode only 1107904 codes, so we are missing somehow 4160 codes that python can't encode from bytes. I won't deal with this for the moment, I'll just believe python that it knows how to encode utf-8 (or I should start creating tests and find if python has a bug and create the ticket ... I must stay strong and follow my goal without diverging as I almost have no time)

In [ ]:
1112064 - 1107904

In [ ]:
128 + (2**5 * 2**6)+ (2**4 * (2**6)**2) + (2**3 * (2**6)**3)

In [ ]:
2**21 + 2**16 + 2**11 + 2**7

In [ ]:
print("indices for the segments: ", 0, 128, (128 + 2**5 * 2**6), (128 + 2**4 * (2**6)**2), (128 + 2**3 * (2**6)**3) )

In [ ]:
# from:  https://stackoverflow.com/questions/7971618/python-return-first-n-keyvalue-pairs-from-dict
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [ ]:
take(50, n2t.items())

In [ ]:
t2n['\x09']

In [ ]:
len(take(10, t2c.items())[0][1])

In [ ]:
import torch
from torch import sparse


In [ ]:
codes = torch.from_numpy(tables[0])

In [ ]:
# from https://discuss.pytorch.org/t/how-to-convert-a-dense-matrix-to-a-sparse-one/7809

def to_sparse(x):
    """ converts dense tensor x to sparse format """
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)

    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())


In [ ]:
scodes = to_sparse(codes)

In [ ]:
scodes.is_sparse

In [ ]:
type(scodes)

In [ ]:
# pytorch sparse can't be saved yet ... not implemented for the moment (I should do it myself and send the patch)
# torch.save(scodes, "utf8-codes.pt")
# save_obj(scodes, "utf8-codes.torch")

In [ ]:
import scipy as sp
import scipy.sparse

In [ ]:
spcodes = sp.sparse.coo_matrix(tables[0])

In [ ]:
save_obj(spcodes, "utf8-codes-scipy-sparse_3seg")

So, for the moment we have the posibility to encode all utf-8 characters, but is still a bit big in size when having the complete. But I'll try to cut the use of memory because 6.8GB for the "dense" matrix reprsentation is too much. In Sparse mode matrix is only 83MB for the entire dataset. Nevertheless there are many characters that I will not be using for the first tests, so having it use only a part will (should) be enough.

So I'll see how big the encoder is without the 4 segments, but only using the first 3 (this should be enough for most applications) so we can encode 

number of codes =  59328

number of code_exceptions =  4224

the entire code is now 206MB on a file on disk in non sparse mode and 3.6MB on sparse mode on disk for codes with 4 segments (this mode is scalable to later add the rest of the code without the need of redoing the architecture)

But also reducing the number of bytes on the code (using only 3 bytes max instead of 4) by not taking the last one that anyways we are not using for this application we can reduce this to 177MB of the complete "dense" code on disk and 3.6MB on sparse mode.

I would not recomend doing this all the time as it restricts the power of the input network to only known elements (and we want to do with all the possible codes) but for my tests this reduces the usage of memory, the amount of parameters and the processing time.

When using 4 segments there are 452 elements per code, when using 3 there are 388

So I can start playing with it without worrying about memory ;)

NOTE:
I have now cut the size of the embedding by 128, this is taking out redundancy from the first segment. 

Also, for the special codes I count on using the first 20 elements of the utf-8 0000 -> 0020 which are special indicators as NUL, SOH, STX, ETX, EOT, ACK, ....

The mapping will be:
- *\<start\>* -> STX = 0002 Start of Transaction
- *\<end\>* -> ETX = 0003 End of Transaction
- *\<unknown\>* = *\<unk\>* -> NAK = 0015


In [ ]:
d = np.load("utf8_code_matrix_3seg.npy")

In [ ]:
d = torch.from_numpy(d)

In [ ]:
cudad = d.cuda()

In [ ]:
len(d)

In [ ]:
# trying the autoencoder now, this is just to see if it works before going on with a more complex setup
import numpy as np
import pickle
import torch
from utf8_encoder import *
from utf8vae import *
from torch.utils.data import TensorDataset

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar, vector_size, channels=1):
#     print("x shape = ", x.shape, recon_x.shape)
#     BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
#     BCE = F.nll_loss(recon_x, x)
    BCE = F.mse_loss(recon_x, x)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


In [ ]:

def train_overfit():
    # generate dataset inputs (basically the same as the encoding)
    # We are going to overfit
    epochs = 100
    segments = 3  # I do with 3 as it will be much MUCH faster and smaller for my resources than 4 segments
    in_size = 388-128  # 3 segments
    hidd_size = 100
    code_size = 50
    device = device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    vector_size = code_size
    channels = 1
    batch_size = 10
    datafile = "utf8_code_matrix_3seg.npy"
    log_interval = 10
    
    model = UTF8VAE(in_size, hidd_size, code_size, segments=segments)
    # loader = DataLoader(UTF8Dataset("utf8_code_matrix_3seg.npy"), batch_size=batch_size)

    name = "utf8-vae-3segments-overfit"
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    n_batches = 10
    n_epocs = 100
    # train_loader, test_loader = get_loaders(batch_size, transformation)
    # we are overfitting, so train and test is the same thing.

#     for epoch in range(1, epochs + 1):
#         train(model, optimizer, loss_function, loader,epoch, vector_size, channels)
#         test(model, loader, epoch, vector_size, channels)
    data = torch.from_numpy(np.load(datafile)).float()
    data = data#.to(device)
    model = model#.to(device)
    
    model.train()
    for epoch in range(n_epocs):
        train_loss = 0
        for batch_idx in range(n_batches):
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_function(recon_batch, data, mu, logvar, vector_size, channels)
            loss.backward()
            train_loss += loss.data.item()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(data),
                    100. * batch_idx / len(data),
                    loss.data.item() / len(data)))

        print('====> Epoch: {} Average loss: {:.4f}'.format(
              epoch, train_loss / len(data)))


    model.save_model(name, "saved_models")
    # train_all(models)


In [ ]:
train_overfit()

In [ ]:
torch.__version__